In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install seaborn


In [1]:
import pandas as pd

import csv
import re # for regular expression
import string
from datetime import datetime

import nltk 
from nltk.tokenize import RegexpTokenizer # dIvide the words 
from collections import Counter #count the number of word for each list 

import numpy as np 
import matplotlib.pyplot as plt

import seaborn


#import plotly
#import plotly.plotly as py
#import plotly.graph_objs as go
#plotly.tools.set_credentials_file(username='XXXXXX', api_key='XXXXXXX')

%matplotlib inline


In [2]:
# change the seeting of the cells to show the entire content of coulmn ( eexpand the size of coulmn)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
# Step 6: ML Classification     

In [4]:
# read csv file and save it in a data frame
data_df3=pd.read_csv('Training data after method1.csv') 

In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.metrics  import confusion_matrix, classification_report
from sklearn import metrics

In [6]:
## remove data with NAN stance
data_df3=data_df3[~data_df3["sentiment"].isna()]

In [7]:
# remove the "Neutral" class
data_df3=data_df3[data_df3['sentiment'] != 'neutral']

In [8]:
data_df3.head(5)

,tweets,len,ID,Date,Source,Likes,RTs,location,UserName,Name,clean tweets,clean tweet length,sentiment
1,RT @noon: بالبسمة 🎶\n\nتوصلك 💛\n\nالبهجة 😁\n\n#طلباتك_أوامر \n#هلا_نون https://t.co/H49FTSoVJj,87,1237056779881713665,2020-03-09 16:44:54,Twitter for Android,0,39,Saudi Arabia,almrkazameen991,Ameen althrhain,RT بالبسمه 🎶 توصلك 💛 البهجه 😁 طلباتكاوامر هلانون,53,positive
2,See Whats Happening Around Town &amp; Worldwide @Noon W/ @SylviaFOX32 &amp; @KaitlinCody With All The Lastest On Your #1 Ne… https://t.co/i22G7k6xtq,148,1237055940840640512,2020-03-09 16:41:34,Twitter for iPhone,0,0,USA,DeeMillChi,DeeMill,Se Whats Hapening Around Town amp Worldwide W amp With Al The Lastest On Your 1 Ne,83,positive
5,@noon عندي رصيد في محفظة نون ابي ارجعها لبطاقتي كيف ؟,53,1237052791610388483,2020-03-09 16:29:03,Twitter for Android,0,0,Saudi Arabia,iika_desi,"كاديس 🖐🔥""",عندي رصيد في محفظه نون ابي ارجعها لبطاقتي كيف,47,negative
10,@noon كيف افعل بطاقة هديه الراجحي,33,1237033937995730950,2020-03-09 15:14:08,Twitter for Android,0,0,None,k4e2w6SSnyKYdb6,الوافي,كيف افعل بطاقه هديه الراجحي,28,negative
13,@noon https://t.co/frPlvVM5tE,29,1237022055683555329,2020-03-09 14:26:55,Twitter for iPhone,0,0,None,fuadzamareeh,Fuad,,1,negative


In [9]:
data_df3['sentiment'] = data_df3['sentiment'].map({'positive': 1,'negative': 0}) 
data_df3.head(5)

,tweets,len,ID,Date,Source,Likes,RTs,location,UserName,Name,clean tweets,clean tweet length,sentiment
1,RT @noon: بالبسمة 🎶\n\nتوصلك 💛\n\nالبهجة 😁\n\n#طلباتك_أوامر \n#هلا_نون https://t.co/H49FTSoVJj,87,1237056779881713665,2020-03-09 16:44:54,Twitter for Android,0,39,Saudi Arabia,almrkazameen991,Ameen althrhain,RT بالبسمه 🎶 توصلك 💛 البهجه 😁 طلباتكاوامر هلانون,53,1
2,See Whats Happening Around Town &amp; Worldwide @Noon W/ @SylviaFOX32 &amp; @KaitlinCody With All The Lastest On Your #1 Ne… https://t.co/i22G7k6xtq,148,1237055940840640512,2020-03-09 16:41:34,Twitter for iPhone,0,0,USA,DeeMillChi,DeeMill,Se Whats Hapening Around Town amp Worldwide W amp With Al The Lastest On Your 1 Ne,83,1
5,@noon عندي رصيد في محفظة نون ابي ارجعها لبطاقتي كيف ؟,53,1237052791610388483,2020-03-09 16:29:03,Twitter for Android,0,0,Saudi Arabia,iika_desi,"كاديس 🖐🔥""",عندي رصيد في محفظه نون ابي ارجعها لبطاقتي كيف,47,0
10,@noon كيف افعل بطاقة هديه الراجحي,33,1237033937995730950,2020-03-09 15:14:08,Twitter for Android,0,0,None,k4e2w6SSnyKYdb6,الوافي,كيف افعل بطاقه هديه الراجحي,28,0
13,@noon https://t.co/frPlvVM5tE,29,1237022055683555329,2020-03-09 14:26:55,Twitter for iPhone,0,0,None,fuadzamareeh,Fuad,,1,0


In [10]:
data_df3.to_csv("Ready Data.csv",index=False)

In [11]:
# idneitfy the data and the labels
data= data_df3['clean tweets']
target= data_df3['sentiment']

In [12]:
# Use TfidfVectorizer for feature extraction (TFIDF to convert textual data to numeric form):
tf_vec = TfidfVectorizer()
X = tf_vec.fit_transform(data)
X.shape

(1780, 5472)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.50, random_state=0)

In [14]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)


(890, 5472) (890, 5472) (890,) (890,)


In [15]:
# Classifier 1: Naive Bayes   

In [16]:
# create the classifer and fit the training data and lables

classifier_nb = MultinomialNB().fit(X_train.todense(),y_train)

print("MultinomialNB accuracy: %.2f"%classifier_nb.score(X_test.todense(), y_test))

#do a 10 fold cross-validation 
results_nb = cross_val_score(classifier_nb, X.todense(),target, cv=10)
print("\n10-fold cross-validation:")
print(results_nb)

print("The average accuracy of the MultinomialNB classifier is : %.2f" % np.mean(results_nb))

print("\nConfusion matrix of the MultinomialNB classifier:")
predicted_nb = classifier_nb.predict(X_test.todense())
print(confusion_matrix(y_test,predicted_nb))


print("\nClassification_report of MultinomialNB classifier:")
print(classification_report(y_test,predicted_nb))
print("----------------------------------------------------------------------------")


MultinomialNB accuracy: 0.90

10-fold cross-validation:
[0.89325843 0.89325843 0.89325843 0.89325843 0.89325843 0.88764045
 0.89325843 0.89325843 0.89325843 0.89325843]
The average accuracy of the MultinomialNB classifier is : 0.89

Confusion matrix of the MultinomialNB classifier:
[[  0  91]
 [  0 799]]

Classification_report of MultinomialNB classifier:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        91
           1       0.90      1.00      0.95       799

    accuracy                           0.90       890
   macro avg       0.45      0.50      0.47       890
weighted avg       0.81      0.90      0.85       890

----------------------------------------------------------------------------


C:\Users\Hadeel\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
#logsitic regrssion   

In [18]:
# build the second classifer, logsitic regrssion 
classifier_log =  LogisticRegression().fit(X_train,y_train)

C:\Users\Hadeel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [19]:
print("Logisitic Accuracy: %.2F"%classifier_log.score(X_test, y_test))
results_log = cross_val_score(classifier_log, X,target, cv=10)


print("\n10-fold cross-validation:")
print(results_log)


print("The average accuracy of the Logisitic classifier is : %.2f" % np.mean(results_log))
print("\nConfusion matrix of the Logisitic classifier:")
predicted_log= classifier_log.predict(X_test)
print(confusion_matrix(y_test,predicted_log))


print("\nClassification_report of Logisitic classifier:")
print(classification_report(y_test,predicted_log))
print("----------------------------------------------------------------------------")

Logisitic Accuracy: 0.90

10-fold cross-validation:
[0.89325843 0.89325843 0.89325843 0.89325843 0.89325843 0.89325843
 0.89325843 0.89325843 0.89325843 0.89325843]
The average accuracy of the Logisitic classifier is : 0.89

Confusion matrix of the Logisitic classifier:
[[  0  91]
 [  0 799]]

Classification_report of Logisitic classifier:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        91
           1       0.90      1.00      0.95       799

    accuracy                           0.90       890
   macro avg       0.45      0.50      0.47       890
weighted avg       0.81      0.90      0.85       890

----------------------------------------------------------------------------


C:\Users\Hadeel\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Hadeel\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
